<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S8_9_retail_analytics/S9-Retail_pricing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q pyomo
from pyomo.environ import *
from pyomo.gdp import *

     |████████████████████████████████| 2.4MB 2.8MB/s 
     |████████████████████████████████| 256kB 25.2MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 163kB 24.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import pickle

In [0]:
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/S8_9_retail_analytics/salesCereals.csv'
salesCereals = pd.read_csv(url)
salesCereals['WEEK_END_DATE'] = pd.to_datetime(salesCereals['WEEK_END_DATE'])
salesCereals['UPC'] = salesCereals['UPC'].astype(str)

productList = list(salesCereals['UPC'].unique())
print(productList)

feature_list = ['PRICE', 'FEATURE', 'DISPLAY','TPR_ONLY','RELPRICE']

#take the point at the date of the end of the training set of UPC[0]
i = 0
productDataSize = len(salesCereals.loc[salesCereals['UPC']==productList[i]][feature_list].values)
train_size = int(productDataSize * 0.70)
print("N.Points[i="+str(i)+"]:" + str(productDataSize)+", N.TrainPoints[" + str(i) + "]:" + str(train_size))
dateReference = salesCereals.iloc[train_size]['WEEK_END_DATE']
print(dateReference)    



['1111085319', '1111085350', '1600027527', '1600027528', '1600027564', '3000006340', '3800031829']
N.Points[i=0]:156, N.TrainPoints[0]:109
2009-04-29 00:00:00


In [0]:
regr = OrderedDict()
for upc in productList:
    filename = './'+str(upc)+'_tree_model.sav'
    # load the model from disk
    regr[upc] = pickle.load(open(filename, 'rb'))

FileNotFoundError: ignored

In [0]:
# get the row of max date value
latestBasePrice = salesCereals.loc[salesCereals['WEEK_END_DATE'] == dateReference][['UPC','BASE_PRICE']]
latestBasePrice.set_index('UPC', inplace=True)
print(latestBasePrice)

            BASE_PRICE
UPC                   
1111085319        1.89
1111085350        2.39
1600027527        3.19
1600027528        4.59
1600027564        3.19
3000006340        2.74
3800031829        3.29


In [0]:
priceStepSize = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

priceOptions = {}

for k in latestBasePrice.index:
    priceOptions[k] = [p*latestBasePrice.loc[k]['BASE_PRICE'] for p in priceStepSize]
    #priceOptions.append([p*latestBasePrice.loc[k]['BASE_PRICE'] for p in priceStepSize])
    #print(type(priceStepSize))

sumPrice = sum(latestBasePrice['BASE_PRICE'].values)
print('sumPrice:' +str(sumPrice))

X_input_pd = []

for upc in productList:
    i = productList.index(upc)
    for p in range(len(priceStepSize)):
        X_input_pd.append([upc]+[priceStepSize[p]]+[priceOptions[upc][p]] + [0,0,0] + [priceOptions[upc][p]/sumPrice])

X_input_pd = pd.DataFrame(X_input_pd,columns = ['UPC', 'OPTION'] + feature_list)
#print(X_input_pd)

X_input = OrderedDict()

for upc in productList:
    X_input[upc] = X_input_pd.loc[X_input_pd['UPC']==upc][feature_list].values
    
    # Predicted sales cannot be negative
    y_pred[upc] = np.maximum(regr[upc].predict(X_input[upc]), 0.0)
    
    print(reversed(range(1, len(y_pred[upc]))))
    
    # N.Units must be a monotonic decreasing function of price
    for ind in reversed(range(1, len(y_pred[upc]))):
        if(y_pred[upc][ind] > y_pred[upc][ind - 1]):
               y_pred[upc][ind - 1] = y_pred[upc][ind]
    
    print('UPC['+str(upc)+']: '+str(y_pred[upc])+"\n")


print(list(X_input.keys()))

sumPrice:21.28
UPC[1111085319]: [19. 19. 19. 19. 19. 19. 19. 19. 19.]

UPC[1111085350]: [24.         24.         24.         24.         24.         13.76190476
 13.76190476 13.76190476 13.76190476]

UPC[1600027527]: [33.      33.      16.21875 16.21875 16.21875 11.5     11.5     11.5
 11.5    ]

UPC[1600027528]: [76. 76. 31. 31. 31. 31. 31. 31. 31.]

UPC[1600027564]: [21.83908046 21.83908046 21.83908046 21.83908046 21.83908046 21.83908046
 21.83908046 21.83908046 21.83908046]

UPC[3000006340]: [24. 24. 24. 24. 24. 24. 24. 24. 24.]

UPC[3800031829]: [26.         26.         26.         26.         25.56818182 25.56818182
 20.         20.         20.        ]

['1111085319', '1111085350', '1600027527', '1600027528', '1600027564', '3000006340', '3800031829']


In [0]:

from pyomo.environ import *
from pyomo.gdp import *

model = ConcreteModel()

print(list(range(len(productList))))

iIndexList = list(range(len(productList)))
jIndexList = list(range(len(priceStepSize)))

# Variables
model.x = Var(productList, jIndexList, within = Binary)

# Objective function
def ObjRule(model):
    return sum(model.x[upc,j]*priceOptions[upc][j]*y_pred[upc][j] \
               for upc in productList for j in jIndexList)
    
model.opt = Objective(rule=ObjRule, sense=maximize)

# Constraints
# Constraints 1: choose one price option
model.allFinish = ConstraintList()
for upc in productList:
    model.allFinish.add(sum(model.x[upc, j] for j in jIndexList)==1)

# Constraints 2: allowable range for the sum of price
model.allFinish.add(sum(model.x[upc,j]*priceOptions[upc][j] \
                    for upc in productList for j in jIndexList) >= sumPrice*0.95)
model.allFinish.add(sum(model.x[upc,j]*priceOptions[upc][j] \
                    for upc in productList for j in jIndexList) <= sumPrice*0.95)

#model.pprint()


[0, 1, 2, 3, 4, 5, 6]


    'pyomo.core.base.constraint.ConstraintList'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


RuntimeError: Cannot add component 'allFinish_index' (type <class 'pyomo.core.base.sets.SimpleSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.sets.SimpleSet'>) is already defined.

In [0]:
#optimizer = SolverFactory('cbc',executable='C:/Program Files (x86)/COIN-OR/1.7.4/win32-msvc9/bin/cbc.exe')
results = optimizer.solve(model)


In [0]:
sequence_tuples = []
totalRevenue = 0
for i in model.x._data:
    #if not model.x._data[i[0],i[1]].value is None:
    if(value(model.x[i[0],i[1]]) > 0.0001):
        sequence_tuples.append((i[1], i[0]))
        print("x["+str(i[0])+","+str(i[1])+"]="+ str(model.x._data[i[0],i[1]].value))
        
        totalRevenue += value(model.x[i[0],i[1]])*priceOptions[i[0]][i[1]]*y_pred[i[0]][i[1]]
        
totalOriginalRevenue = sum(priceOptions[upc][priceStepSize.index(1.0)]*y_pred[i[0]][priceStepSize.index(1.0)] \
                                for upc in productList)
        
print('OBJ (optimized):'+"{0:.2f}".format(totalRevenue))
print('OBJ (original):'+"{0:.2f}".format(totalOriginalRevenue))
print('Revenue Increase:' + "{0:.2f}".format((totalRevenue-totalOriginalRevenue)/totalOriginalRevenue*100)+"%")

x[1111085319,8]=1.0
x[1111085350,1]=1.0
x[1600027527,1]=1.0
x[1600027528,1]=1.0
x[1600027564,8]=1.0
x[3000006340,8]=1.0
x[3800031829,7]=1.0
OBJ (optimized):608.58
OBJ (original):544.09
Revenue Increase:11.85%
